In [6]:
pip install requests beautifulsoup4 eland elasticsearch huggingface-hub langchain tqdm torch requests sentence_transformers googletrans==4.0.0-rc1

Note: you may need to restart the kernel to use updated packages.



# Web Scraping con BeautifulSoup

Este cuaderno demuestra cómo realizar web scraping utilizando `requests` y `BeautifulSoup` para extraer el contenido principal de un artículo de una página web, formatearlo en HTML similar a la página original y agregar una referencia a la fuente.

## Pasos:
1. **Enviar una solicitud HTTP**: Utilizamos `requests` para enviar una solicitud HTTP a la URL de la página web.
2. **Analizar el HTML**: Usamos `BeautifulSoup` para analizar el contenido HTML de la página web.
3. **Identificar y extraer el contenido**: Buscamos los elementos HTML que contienen el contenido principal del artículo (párrafos y encabezados).
4. **Formatear el contenido en HTML**: Estructuramos el contenido extraído en una plantilla HTML.
5. **Guardar el contenido en un archivo**: Guardamos el contenido formateado en un archivo HTML para su visualización.

A continuación, se muestra el código que implementa estos pasos:


In [16]:
web = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6627879/"

In [17]:
import requests
from bs4 import BeautifulSoup

def beautifulsoup_web_scrape_url(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Verifica si la solicitud fue exitosa
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener la página web: {e}")
        return None

def extract_main_content(soup):
    # Definir posibles contenedores de contenido
    possible_containers = ['article', 'div', 'main', 'section']

    for tag in possible_containers:
        main_content = soup.find(tag)
        if main_content:
            paragraphs = main_content.find_all(['p', 'h2', 'h3'])
            if paragraphs:
                content_html = ""
                for section in paragraphs:
                    content_html += f"<{section.name}>{section.get_text()}</{section.name}>\n"
                return content_html

    # Si no se encuentra un contenedor claro, buscar todos los párrafos
    paragraphs = soup.find_all('p')
    if paragraphs:
        content_html = ""
        for p in paragraphs:
            content_html += f"<p>{p.get_text()}</p>\n"
        return content_html

    return "No se pudo encontrar el contenido del artículo."

url = web
soup = beautifulsoup_web_scrape_url(url)

if soup:
    article_html = extract_main_content(soup)
    if article_html:
        formatted_html = f'''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Artículo</title>
        </head>
        <body>
            {article_html}
            <footer>
                <p>Fuente: <a href="{url}">{url}</a></p>
            </footer>
        </body>
        </html>
        '''
        with open("article.html", "w", encoding="utf-8") as file:
            file.write(formatted_html)
        print("El contenido del artículo ha sido formateado y guardado en 'article.html'.")
    else:
        print("No se pudo extraer el contenido del artículo.")
else:
    print("Error al obtener la página web.")


El contenido del artículo ha sido formateado y guardado en 'article.html'.
